In [1]:
pip install colorama

In [3]:
import requests, urllib, json, colorama
from bs4 import BeautifulSoup
from colorama import init, Fore, Style

init(autoreset=True)

sorgu = "inurl:udemy.com/course/ \"inurl:couponCode"
# Google Search Part
tbs = "d" 
URL = f"https://google.com/search?q={sorgu}&tbs=qdr:{tbs}"


# Create a vir searching
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
ACCEPT_ENCODING = "gzip, deflate, br"

headers = { 'User-Agent' : USER_AGENT, 'Content-Type': "text/plain" }

resp = requests.get(URL, headers=headers)
try:
  #If we connect after we split page with BeautifulSoup
  if resp.status_code == 200:
      soup = BeautifulSoup(resp.content, "html.parser")

  link, title, coupon = [], [], []
  for g in soup.find_all('div', class_='r'):
      anchors = g.find_all('a')
      if anchors:
          title.append(g.find('h3').text)
          link.append(anchors[0]['href'])
          coupon.append((anchors[0]['href']).partition("?couponCode=")[2]) 

  results = [{"Title": t, "Link": l, "Coupon": c} for t, l, c in zip(title, link, coupon)]

  for i in range(len(results)):

      resp = requests.get(results[i]["Link"], headers=headers)

      if resp.status_code == 200:
          course_code = BeautifulSoup(resp.content, "html.parser").body['data-clp-course-id']
        
          CodeCheck_URL = f"https://www.udemy.com/api-2.0/course-landing-components/{course_code}/me/?components=buy_button,purchase,redeem_coupon,discount_expiration,gift_this_course&discountCode="+results[i]["Coupon"]

          resp = requests.request("GET",CodeCheck_URL, headers=headers).json()
        
        # Write consol on coupons and courses
          if (resp)["purchase"]["data"]["pricing_result"]["price"]["amount"] == 0:
              print(f"\n[{Fore.GREEN}*{Style.RESET_ALL}]")
              print(f"\t{Fore.GREEN}Başlık: {Style.RESET_ALL}"+results[i]["Title"], sep='\n')
              print(f"\t{Fore.GREEN}Adres: {Style.RESET_ALL}"+results[i]["Link"], sep='\n')
              print(f"\t{Fore.GREEN}Kupon: {Style.RESET_ALL}"+results[i]["Coupon"]+ 
                  f"\t\t{Fore.GREEN}Tahmini Kalan Kupon: {Style.RESET_ALL}"+str((resp)["purchase"]["data"]["pricing_result"]["campaign"]["uses_remaining"])
                  +str(f"\t\t{Fore.GREEN}Kupon Bitiş Zamanı:: {Style.RESET_ALL}"+str((resp)["purchase"]["data"]["pricing_result"]["campaign"]["end_time"])[:-9])) 
             
except KeyError:
  pass
      


[*]
	Başlık: Financial Modelling, Financial Planning and DCF Valuation ...
	Adres: https://www.udemy.com/course/financial-modelling-financial-planning-and-dcf-valuation/?couponCode=SPECIALOFFER
	Kupon: SPECIALOFFER		Tahmini Kalan Kupon: 39779		Kupon Bitiş Zamanı:: 2020-04-10 19:52

[*]
	Başlık: ISC SSCP Cryptography Certified Practice Exam | Udemy
	Adres: https://www.udemy.com/course/isc-sscp-cryptography-certified-practice-exam/?couponCode=ISC-105
	Kupon: ISC-105		Tahmini Kalan Kupon: 39392		Kupon Bitiş Zamanı:: 2020-04-10 12:19
